In [201]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [202]:
df_r = pd.read_csv('record_corrected3.csv', low_memory=False).drop('Unnamed: 29', axis=1)
df_s = pd.read_csv('ship_final.csv', low_memory=False).rename(columns={'correct.ship_index': 'ship_index'}).drop(columns=['Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13'])

In [203]:
df_s['ship_name'] = df_s['ship_name'].str.strip().str.title()
df_s['ship_code'] = df_s['ship_code'].str.strip().str.upper()
df_s['ship_index'] = df_s['ship_index'].astype(str)
df_r['ship_index'] = df_r['ship_index'].astype(str)

In [204]:
for index, row in df_r.iterrows():
    if '/' in row['ship_index']:
        if row['ship_index'].split('/')[0] == '':
            df_r.at[index, 'ship_index'] = row['ship_index'].split('/')[1].strip()
        else:
            temp = row['ship_index'].split('/')
            df_r.at[index, 'ship_index'] = temp[0].strip()
for index, row in df_s.iterrows():
    if '/' in row['ship_index']:
        if row['ship_index'].split('/')[0] == '':
            df_s.at[index, 'ship_index'] = row['ship_index'].split('/')[1].strip()
        else:
            temp = row['ship_index'].split('/')
            df_s.at[index, 'ship_index'] = temp[0].strip() + '/' + temp[1].strip()

In [205]:
sind_r = list(set(df_r['ship_index']))
sind_s = list(set(df_s['ship_index']))
absents = []
for i in sind_r:
    if i not in sind_s:
        absents.append(i)

In [206]:
mapper = {}
mapper_s = {}
pp = ['0/32',
 '151/154',
 '161/167',
 '163/325',
 '288/289',
 '292/293',
 '316/317',
 '50/61',
 '60/295']
for i in pp:
    oh = i.split('/')
    mapper[oh[0]] = oh[1]
for i in pp:
    mapper_s[i] = i.split('/')[1]
mapper_s['247'] = '248'
mapper['247'] = '248'

In [207]:
df_r['ship_index'] = df_r['ship_index'].replace(mapper)
df_s['ship_index'] = df_s['ship_index'].replace(mapper_s)

In [208]:
sind_r = list(set(df_r['ship_index']))
sind_s = list(set(df_s['ship_index']))
absents = []
for i in sind_s:
    if i not in sind_r:
        absents.append(i)

In [212]:
df_s[df_s['ship_index'].isin(absents)].to_csv('still_absent_ships.csv')

In [237]:
df_s.to_csv('ship_rev.csv')
df_r.to_csv('record_rev.csv')

In [238]:
df_r_columns = list(df_r.columns)
df_p = pd.read_csv('person.csv', low_memory=False).rename(columns={'id': 'person_id'})
df_p_columns = list(df_p.columns)
df_v = pd.read_csv('voyage.csv', low_memory=False).rename(columns={'id': 'voyage_id'}).drop(columns=['ship_code'])
df_v_columns = list(df_v.columns)
df_s_columns = list(df_s.columns)
df_f = pd.read_csv('functions.csv', low_memory=False).rename(columns={'id': 'function_id'})
df_f_columns = list(df_f.columns)
df_o = pd.read_csv('origin.csv', low_memory=False).rename(columns={'id': 'origin_id'})
df_o_columns = list(df_o.columns)
df1 = pd.merge(df_r, df_s, on='ship_index', how='inner')
df2 = pd.merge(df1, df_v, on='voyage_index', how='inner')
df3 = pd.merge(df2, df_p, on='person_index', how='inner')
df4 = pd.merge(df3, df_o, on='origin_index', how='inner')
df = pd.merge(df4, df_f, on='function_index', how='inner')